In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import HTML
from newspaper import Article
from datetime import datetime, date, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings("ignore")
from selenium.webdriver.common.by import By
import time
from joblib import Parallel, delayed
import tqdm 
from tqdm import tqdm

In [2]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

PATH=ChromeDriverManager().install()

In [3]:
df_noticias = pd.read_excel('../data/noticias_diarios_deportivos.xlsx')
df_noticias.drop(['Unnamed: 0', 'level_0', 'index'], axis=1, inplace=True)
df_noticias.head()

,web,titular,redactor,comentarios,link,noticia,seccion,fecha_publicacion,fecha_actual,desactualizacion,equipo,Género_Redactor
0,Mundo D.,Robo de bicicletas antes de comenzar el cicloc...,Agencias,1,https://www.mundodeportivo.com//ciclismo/20230...,Noticia,ciclismo,2023-01-22 00:00:00,2023-01-23,1,otro,desconocido
1,Mundo D.,"Jon Rahm, nº1 en FedEx Cup pero nº3 de un polé...",Redaccion,1,https://www.mundodeportivo.com//golf/20230123/...,Noticia,golf,2023-01-23 00:00:00,2023-01-23,0,otro,desconocido
2,Mundo D.,"Jon Rahm: ""Aprendí de la ética de trabajo de K...",Redaccion,0,https://www.mundodeportivo.com//golf/20230123/...,Noticia,golf,2023-01-23 00:00:00,2023-01-23,0,otro,desconocido
3,Mundo D.,"Los resultados de hoy lunes, octavos de final ...",Mundodeportivo.Com,0,https://www.mundodeportivo.com//tenis/open-aus...,Noticia,tenis,2023-01-23 00:00:00,2023-01-23,0,otro,desconocido
4,Mundo D.,26-28: Primera derrota de España en el Mundial...,Lluis,0,https://www.mundodeportivo.com//balonmano/2023...,Noticia,balonmano,2023-01-22 00:00:00,2023-01-23,1,otro,hombre


In [4]:
cuenta_twitter = ['marca' if w == 'Marca' else'diarioas' if w == 'AS' else'mundodeportivo' if w == 'Mundo D.' else
    'sport' if w == 'SPORT' else'Estadio_ED' if w == 'Estadio D.' else'superdeporte_es'for w in df_noticias['web']]

In [5]:
df_noticias['@twitter'] = cuenta_twitter

In [6]:
busca_twitter = ['https://twitter.com/search?q='+li+'&src=typed_query&f=top' for li in df_noticias['link']]

In [7]:
df_noticias['busca_twitter'] = busca_twitter

In [8]:
# Open Twitter URL
driver = webdriver.Chrome(PATH, options=opciones)

In [9]:
lst_contenido = []
lst_link = []
lst_publi = []
lst_date = []
lst_stats = []
lst_alcance = []
lst_encontrado = []

In [10]:
df_tweets = pd.DataFrame()

In [13]:
for i,bu in enumerate(df_noticias['busca_twitter'][0:2501]):
    
    print(f"Buscando noticia nº {i}, link: {df_noticias['link'][i]}")
    
    try:
        
        driver.get(bu)
        time.sleep(2)
        body = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/section/div')
        time.sleep(2)
        tweets = driver.find_elements(By.CSS_SELECTOR, "[data-testid='tweet']")
        time.sleep(2)

        for tweet in tweets:
            try:
                contenido = tweet.text.replace('\n',' ')
                lst_contenido.append(contenido)

                if ' Promocionado' in contenido:
                    lst_publi.append(True)
                else:
                    lst_publi.append(False)
            except:
                lst_contenido.append('desconocido')
                lst_publi.append(False)

            try:
                stats = tweet.get_attribute("innerHTML").split(' role="group" class="css-1dbjc4n')[0].split('<div aria-label=')[-1].replace('"','')
                lst_stats.append(stats)
            except:
                lst_stats.append('desconocido')

            try:
                alcance = tweet.find_elements(By.CSS_SELECTOR, 'span[data-testid="app-text-transition-container"]')[3].text
                lst_alcance.append(alcance)
            except:
                alcance = 'desconocido'
                lst_alcance.append('desconocido')
                
            lst_link.append(df_noticias['link'][i])
            lst_encontrado.append(True)
        
    except:
        
        lst_link.append(df_noticias['link'][i])
        lst_contenido.append('desconocido')
        lst_publi.append('desconocido')
        lst_stats.append('desconocido')
        lst_alcance.append('desconocido')
        lst_encontrado.append(False)
        
df_tweets['link'] = lst_link
df_tweets['contenido'] = lst_contenido
df_tweets['publi'] = lst_publi
df_tweets['stats'] = lst_stats
df_tweets['alcance'] = lst_alcance
df_tweets['encontrado'] = lst_encontrado

Buscando noticia nº 0, link: https://www.mundodeportivo.com//ciclismo/20230122/1001923791/robo-bicicletas-comenzar-ciclocross-benidorm.html
Buscando noticia nº 1, link: https://www.mundodeportivo.com//golf/20230123/1001924168/jon-rahm-no1-fedex-cup-no3-mundo-pese-forma.html
Buscando noticia nº 2, link: https://www.mundodeportivo.com//golf/20230123/1001924188/jon-rahm-aprendi-etica-trabajo-kobe-bryant.html
Buscando noticia nº 3, link: https://www.mundodeportivo.com//tenis/open-australia/20230123/1001924189/resultados-hoy-lunes-octavos-final-open-australia.html
Buscando noticia nº 4, link: https://www.mundodeportivo.com//balonmano/20230122/1001924071/espana-francia.html
Buscando noticia nº 5, link: https://www.mundodeportivo.com//hockey-hierba/20230122/1001923781/redsticks-cuatros-final-gracias-shoots-outs.html
Buscando noticia nº 6, link: https://www.mundodeportivo.com//otros-deportes/20230123/1001924203/stefano-scarpitti-nuevo-running-back-coach-barcelona-dragons.html
Buscando noticia 

Buscando noticia nº 56, link: https://www.mundodeportivo.com//elrecomendador/bricolaje/20230122/1001923456/consigue-este-maletin-bosch-de-103-piezas-para-taladrar-y-atornillar-por-29-euros.html
Buscando noticia nº 57, link: https://www.mundodeportivo.com//elrecomendador/moda/20230121/1001923079/repasamos-mejores-ofertas-gafas-hawkers-60-descuento.html
Buscando noticia nº 58, link: https://www.mundodeportivo.com//elrecomendador/amazon/20230121/1001923067/deja-impecable-la-luna-de-tu-coche-con-este-limpiacristales-antilluvia-por-solo-8-euros.html
Buscando noticia nº 59, link: https://www.mundodeportivo.com//elotromundo/gente/20230122/1001923704/ultimo-viaje-aitana-sebastian-yatra-vez-solas-sigue-acrecentando-rumores.html
Buscando noticia nº 60, link: https://www.mundodeportivo.com//elotromundo/television/20230121/1001923423/froilan-rafa-mora-envueltos-discusion-discoteca-rafa-pelota-dct.html
Buscando noticia nº 61, link: https://www.mundodeportivo.com//elotromundo/20230122/1001920064/que

Buscando noticia nº 111, link: https://www.mundodeportivo.com//actualidad/20230122/1001923651/encuentran-mas-restos-humanos-ecoparque-toledo-busca-angel.html
Buscando noticia nº 112, link: https://www.mundodeportivo.com//actualidad/20230122/1001923800/jeremy-renner-revela-rompio-mas-30-huesos-grave-accidente-dct.html
Buscando noticia nº 113, link: https://www.mundodeportivo.com//actualidad/20230122/1001923645/adios-cuentas-compartidas-netflix-cuando-comenzara-cobrar-extra-dct.html
Buscando noticia nº 114, link: https://www.mundodeportivo.com//actualidad/20230123/1001924170/son-portadas-prensa-generalista-hoy-domingo-23-enero-dct.html
Buscando noticia nº 115, link: https://www.mundodeportivo.com//vaya-mundo/kings-league/20230122/1001923898/son-resultados-jornada-4-kings-league-gerard-pique-dct.html
Buscando noticia nº 116, link: https://www.mundodeportivo.com//actualidad/20230122/1001923675/bill-gates-habla-alto-claro-sobre-cual-sera-tecnologia-futuro-metaverso-dct.html
Buscando noticia

Buscando noticia nº 175, link: https://www.compramejor.es/10-accesorios-para-sacarle-todo-el-partido-a-tu-freidora-de-aire/
Buscando noticia nº 176, link: https://www.codigonuevo.com/entretenimiento/ciencia/los-foliculos-cultivados-en-laboratorio-podrian-acabar-con-la-calvicie-YY1111818
Buscando noticia nº 177, link: https://www.compramejor.es/mellerware-calefactor-bajo-consumo-barato/
Buscando noticia nº 178, link: https://neomotor.epe.es/coches/el-v12-de-lamborghini-su-historia-antes-de-la-hibridacion-MC1127284
Buscando noticia nº 179, link: https://neomotor.epe.es/industria/volkswagen-abre-el-proceso-de-contratacion-para-la-gigafactoria-de-sagunto-CJ1128242
Buscando noticia nº 180, link: https://neomotor.epe.es/actualidad/las-polemicas-mas-sonadas-de-jeremy-clarkson-CI1129675
Buscando noticia nº 181, link: https://neomotor.epe.es/electricos/vanwall-racing-revive-con-el-deportivo-electrico-vandervell-GB1131833
Buscando noticia nº 182, link: https://ajuntament.barcelona.cat/esports/ca

Buscando noticia nº 241, link: https://www.sport.es/es/noticias/aplicaciones/twitter-cierra-oficialmente-aplicaciones-oficiales-81641204
Buscando noticia nº 242, link: https://www.sport.es/es/noticias/sociedad/cinco-anos-espera-cirugia-trans-81717689
Buscando noticia nº 243, link: https://www.sport.es/es/noticias/sociedad/gorro-frio-caida-pelo-quimio-81718057
Buscando noticia nº 244, link: https://www.sport.es/es/noticias/medio-ambiente/jubilado-93-anos-patenta-aerogenerador-81285516
Buscando noticia nº 245, link: https://www.sport.es/es/noticias/sociedad/gorro-frio-caida-pelo-quimio-81713849
Buscando noticia nº 246, link: https://www.sport.es/es/noticias/cultura/clasificacion-fms-peru-2022-13353084
Buscando noticia nº 247, link: https://www.sport.es/es/noticias/sucesos/amy-fitzpatrick-desaparecida-mijas-restos-hipodromo-81705436
Buscando noticia nº 248, link: https://www.sport.es/es/noticias/nacional/camps-enfrenta-anos-medio-carcel-81712680
Buscando noticia nº 249, link: https://www.

...

Buscando noticia nº 2207, link: https://www.sport.es/es/noticias/apuestas-deportivas/el-corner-de-pronosticos/futbolistas-valorados-firmar-club-81305478
Buscando noticia nº 2208, link: https://www.sport.es/es/noticias/apuestas-deportivas/top-apuestas/rakitic-81358033
Buscando noticia nº 2209, link: https://www.sport.es/es/noticias/apuestas-deportivas/fichaje-rompe-hegemonia-laliga-81340857
Buscando noticia nº 2210, link: https://www.sport.es/es/noticias/nba/clasico-nba-caos-paris-81374572
Buscando noticia nº 2211, link: https://www.sport.es/es/noticias/nba/hawks-fulminan-mavericks-81379246
Buscando noticia nº 2212, link: https://www.sport.es/es/noticias/nba/memphis-amplia-once-racha-victorias-81379090
Buscando noticia nº 2213, link: https://www.sport.es/es/noticias/sociedad/administracion-empresas-impulsar-gases-renovables-80327898
Buscando noticia nº 2214, link: https://www.sport.es/es/noticias/sociedad/puedes-conocer-facilmente-nivel-riesgo-81278472
Buscando noticia nº 2215, link: ht

Buscando noticia nº 2266, link: https://www.mundodeportivo.com//futbol/betis/20230119/1001921641/betis-perdido-80-tandas-penaltis-casa.html
Buscando noticia nº 2267, link: https://www.mundodeportivo.com//futbol/rcd-espanyol/20230118/1001920822/pierre-gabriel-segundo-fichaje-perico.html
Buscando noticia nº 2268, link: https://www.mundodeportivo.com//futbol/girona-fc/20230118/1001921336/tsygankov-guerra-ver-mi-fichaje.html
Buscando noticia nº 2269, link: https://www.mundodeportivo.com//futbol/fc-barcelona/20230118/1001921148/torrejon-polemica-madrid-veces-mas-cosa-vuestra.html
Buscando noticia nº 2270, link: https://www.mundodeportivo.com//futbol/rcd-espanyol/20230118/1001921532/joan-garcia-reivindica-palos.html
Buscando noticia nº 2271, link: https://www.mundodeportivo.com//futbol/athletic-bilbao/20230119/1001921228/reportaje.html
Buscando noticia nº 2272, link: https://www.mundodeportivo.com//futbol/betis/20230119/1001921693/fekir-juanmi-juan-cruz-luiz-felipe-bajas-espanyol.html
Buscan

Buscando noticia nº 2320, link: https://www.mundodeportivo.com//balonmano/20230118/1001921517/espana-polonia.html
Buscando noticia nº 2321, link: https://www.sport.es/es/noticias/sucesos/amenaza-muerte-pareja-atrinchera-machete-gran-canaria-81337315
Buscando noticia nº 2322, link: https://www.mundodeportivo.com//baloncesto/nba/20230119/1001921639/derrota-dallas-ultimatum-luka-doncic.html
Buscando noticia nº 2323, link: https://www.mundodeportivo.com//futbol/fc-barcelona/20230119/1001921368/rashford-gran-peligro-barca-pudo-barca.html
Buscando noticia nº 2324, link: https://www.mundodeportivo.com//futbol/fc-barcelona/20230119/1001921581/clasico-media-supercopa-espana-femenina-juego.html
Buscando noticia nº 2325, link: https://www.mundodeportivo.com//futbol/fc-barcelona/20230118/1001921526/barca-anuncia-fichaje-argentino-lucas-roman.html
Buscando noticia nº 2326, link: https://www.mundodeportivo.com//futbol/copa-del-rey/20230118/1001921596/osasuna-toma-revancha-betis-18-anos.html
Buscando

Buscando noticia nº 2381, link: https://www.sport.es/es/videos/internacional/20230119/jornada-huelga-protestas-francia/81382708.shtml
Buscando noticia nº 2382, link: https://www.sport.es/es/videos/copa-del-rey/20230118/osasuna-celebro-alto-pase-cuartos/81374324.shtml
Buscando noticia nº 2383, link: https://www.sport.es/es/noticias/nieve/atomic-she-skis-mujeres-protagonistas-81244388
Buscando noticia nº 2384, link: https://www.sport.es/es/noticias/nieve/naturland-esqui-nordico-andorra-81209030
Buscando noticia nº 2385, link: https://www.sport.es/es/noticias/bundesliga/bayern-cierra-fichaje-yann-sommer-81353218
Buscando noticia nº 2386, link: https://www.sport.es/es/noticias/mls/inter-miami-ficha-josef-martinez-81361164
Buscando noticia nº 2387, link: https://www.sport.es/es/noticias/mundial-futbol/messi-mes-despues-ganar-mundial-81358515
Buscando noticia nº 2388, link: https://www.sport.es/es/noticias/calcio/dino-baggio-asustado-muertes-vialli-81355249
Buscando noticia nº 2389, link: ht

Buscando noticia nº 2450, link: https://as.com/futbol/mas_futbol/el-depor-da-luz-verde-a-la-cesion-de-retuerta-al-hercules-n/
Buscando noticia nº 2451, link: https://as.com/meristation/2023/01/17/betech/1673971634_977627.html
Buscando noticia nº 2452, link: https://as.com/futbol/segunda/la-ponferradina-cierra-el-fichaje-de-chakla-n/
Buscando noticia nº 2453, link: https://as.com/futbol/femenino/un-superclasico-por-la-gran-final-n/
Buscando noticia nº 2454, link: https://as.com/futbol/femenino/el-ansia-de-olga-marca-el-camino-n/
Buscando noticia nº 2455, link: https://as.com/futbol/femenino/bernabe-mete-a-la-real-en-otra-superfinal-n/
Buscando noticia nº 2456, link: https://as.com/meristation/2023/01/18/noticias/1674029683_166632.html
Buscando noticia nº 2457, link: https://as.com/meristation/2023/01/18/noticias/1674032726_555829.html
Buscando noticia nº 2458, link: https://as.com/meristation/2023/01/17/noticias/1673963613_406015.html
Buscando noticia nº 2459, link: https://as.com/opini

In [14]:
df_tweets

,link,contenido,publi,stats,alcance,encontrado
0,https://www.mundodeportivo.com//ciclismo/20230...,desconocido,desconocido,desconocido,desconocido,False
1,https://www.mundodeportivo.com//golf/20230123/...,desconocido,desconocido,desconocido,desconocido,False
2,https://www.mundodeportivo.com//golf/20230123/...,desconocido,desconocido,desconocido,desconocido,False
3,https://www.mundodeportivo.com//tenis/open-aus...,desconocido,desconocido,desconocido,desconocido,False
4,https://www.mundodeportivo.com//balonmano/2023...,desconocido,desconocido,desconocido,desconocido,False
...,...,...,...,...,...,...
3984,https://as.com/futbol/videos/la-imagen-es-trem...,Juan @Juan35232299 · 20 ene. La imagen es trem...,False,,6,True
3985,https://as.com/actualidad/asi-se-cubrira-espan...,Aitor Igartua Perosanz @AitorPerosanz · 19 ene...,False,,6,True
3986,https://as.com/actualidad/la-persona-mas-vieja...,"luis manuel suárez @luismanuel_cmf · 21 ene. ""...",False,,75,True
3987,https://as.com/actualidad/el-gobierno-anticipa...,Albapaola @palmcf · 19 ene. El Gobierno antici...,False,,46,True


In [15]:
df_tweets.to_excel('../data/tweets_p1.xlsx')

In [16]:
# df_t1 = pd.read_excel('../data/tweets_p1.xlsx')
# df_t2 = pd.read_excel('../data/tweets_p2.xlsx')
# df_t3 = pd.read_excel('../data/tweets_p3.xlsx')
# df_t4 = pd.read_excel('../data/tweets_p4.xlsx')

In [17]:
# df_tweets = pd.concat([df_t1, df_t2, df_t3, df_t4])

In [35]:
# df_tweets.drop('Unnamed: 0', axis=1, inplace=True)
# df_tweets.drop('index', axis=1, inplace=True)
df_tweets.reset_index(inplace=True, drop = True)

### Limpiamos los datos

In [36]:
df_tweets.head()

,link,contenido,publi,stats,alcance,encontrado
0,https://www.mundodeportivo.com//ciclismo/20230...,desconocido,desconocido,desconocido,desconocido,False
1,https://www.mundodeportivo.com//golf/20230123/...,desconocido,desconocido,desconocido,desconocido,False
2,https://www.mundodeportivo.com//golf/20230123/...,desconocido,desconocido,desconocido,desconocido,False
3,https://www.mundodeportivo.com//tenis/open-aus...,desconocido,desconocido,desconocido,desconocido,False
4,https://www.mundodeportivo.com//balonmano/2023...,desconocido,desconocido,desconocido,desconocido,False
...,...,...,...,...,...,...
15624,https://www.sport.es/es/noticias/internacional...,Julio @mrtjulio · 4 ene. Vomitivo sport.es Dec...,False,NaN,6,True
15625,https://www.sport.es/es/noticias/internacional...,alb035ertoᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠᅠ...,True,1 Me gusta,"78,2 mil",True
15626,https://www.sport.es/es/noticias/valladolid/ro...,"Diario SPORT @sport · 3 ene. #RealMadrid ""L...",False,"2 Retweets, 25 Me gusta","10,6 mil",True
15627,https://www.sport.es/es/noticias/real-madrid/c...,Diario SPORT @sport · 3 ene. #RealMadrid #LaL...,False,"4 respuestas, 1 Retweet, 18 Me gusta","11,9 mil",True


In [37]:
df_tweets.to_excel('../data/tweets_selenium.xlsx', index=False)

In [149]:
# for i,s in enumerate(df_tweets['stats']):
#     print(i,s)

In [78]:
likes = []
for s in df_tweets['stats']:
    try:
        if 'Me gusta' in s:
            likes.append(int(s.split(' Me gusta')[0].split()[-1]))
        else:
            likes.append(0)
    except:
        likes.append(0)

In [79]:
rt = []
for s in df_tweets['stats']:
    try:
        if 'Retweets' in s:
            rt.append(int(s.split(' Retweets')[0].split()[-1]))
        else:
            rt.append(0)
    except:
        rt.append(0)

In [80]:
resp = []
for s in df_tweets['stats']:
    try:
        if 'respuestas' in s:
            resp.append(int(s.split(' respuestas')[0].split()[-1]))
        else:
            resp.append(0)
    except:
        resp.append(0)

In [81]:
len(likes), len(rt), len(resp)

(15629, 15629, 15629)

In [82]:
df_tweets['likes'] = likes
df_tweets['rt'] = rt
df_tweets['respuestas'] = resp

In [131]:
alcance = []
for s in df_tweets['alcance']:
    try:
        if 'M' in s:
            alcance.append(int(s.split()[0].split(',')[0]+s.split()[0].split(',')[1]+'00000'))
        elif ',' in s:
            alcance.append(int(s.split()[0].split(',')[0] + s.split()[0].split(',')[1] + '00'))
        
        elif 'desconocido' in s:
            alcance.append(0)
        elif 'mil' in s:
            alcance.append(int(s.split()[0]+'000'))
        else:
            alcance.append(int(s.replace('.','')))
    except:
        alcance.append(0)

In [141]:
# for i,e in enumerate(df_tweets['alcance']):
#     print(f'{e} ----- {alcance[i]}')

In [142]:
df_tweets['alcance'] = alcance

In [153]:
df_tweets['publi'] = [False if pu == 'desconocido' else pu for pu in df_tweets['publi']]

In [160]:
df_tweets = df_tweets[df_tweets['publi']!=True]
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15213 entries, 0 to 15628
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   link        15213 non-null  object
 1   contenido   15213 non-null  object
 2   publi       15213 non-null  bool  
 3   stats       10319 non-null  object
 4   alcance     15213 non-null  int64 
 5   encontrado  15213 non-null  bool  
 6   likes       15213 non-null  int64 
 7   rt          15213 non-null  int64 
 8   respuestas  15213 non-null  int64 
dtypes: bool(2), int64(4), object(3)
memory usage: 980.5+ KB


In [174]:
df_tweets = df_tweets.groupby('link').sum()[['alcance','encontrado','likes','rt','respuestas']].reset_index()

In [178]:
df_tweets.rename({'encontrado':'tweets'}, axis = 1, inplace=True)
df_tweets

,link,alcance,tweets,likes,rt,respuestas
0,http://as.com/futbol/2022/12/31/reportajes/167...,0,0,0,0,0
1,https://ajuntament.barcelona.cat/esports/ca/no...,0,0,0,0,0
2,https://apuestas.as.com/noticias/apuestas-mund...,0,0,0,0,0
3,https://apuestas.as.com/noticias/apuestas-mund...,0,0,0,0,0
4,https://apuestas.as.com/noticias/apuestas-mund...,0,0,0,0,0
...,...,...,...,...,...,...
9716,https://www.superdeporte.es/valencia-cf/,0,1,0,0,0
9717,https://www.superdeporte.es/valencia-cf/2023/0...,5661,5,8,0,0
9718,https://www.superdeporte.es/valencia-cf/2023/0...,6094,5,0,0,0
9719,https://www.superdeporte.es/valencia-cf/2023/0...,8779,5,10,2,6


In [179]:
df_tweets.to_excel('../data/twitter.xlsx')

In [189]:
merged_df = pd.merge(df_noticias, df_tweets, on='link', how='outer')

In [203]:
# merged_df.drop(['@twitter','busca_twitter'], axis=1, inplace=True)
merged_df.rename(columns={"Género_Redactor": "genero_redactor"}, inplace=True)

In [207]:
merged_df['repercusion_twitter'] = merged_df['likes'] + merged_df['rt'] + merged_df['respuestas']

In [209]:
merged_df.tail()

,web,titular,redactor,comentarios,link,noticia,seccion,fecha_publicacion,fecha_actual,desactualizacion,equipo,genero_redactor,alcance,tweets,likes,rt,respuestas,repercusion_twitter
9716,SPORT,El inesperado éxito de un diputado gallego que...,Angel,0,https://www.sport.es/es/noticias/nacional/ines...,Noticia,no deportivas,2023-01-04 00:00:00,2023-01-04,0,otro,hombre,2,1,0,0,0,0
9717,SPORT,Decenas de miles de asesinos y violadores ruso...,Marc,0,https://www.sport.es/es/noticias/internacional...,Noticia,no deportivas,2023-01-04 00:00:00,2023-01-04,0,otro,hombre,15,2,0,0,0,0
9718,SPORT,Ronaldo condena los insultos racistas de la af...,SPORT.es,0,https://www.sport.es/es/noticias/valladolid/ro...,Noticia,futbol,2023-01-03 00:00:00,2023-01-04,1,Valladolid,desconocido,10600,1,25,2,0,27
9719,SPORT,Competición inicia un procedimiento al Real Va...,Sergi,0,https://www.sport.es/es/noticias/real-madrid/c...,Noticia,futbol,2023-01-03 00:00:00,2023-01-04,1,Real Madrid,hombre,11900,1,18,0,4,22
9720,AS,"Padre, agente e ídolo",Mario,10,https://as.com/futbol/primera/el-padre-de-bell...,Noticia,futbol,Desconocido,2023-01-04,Desconocido,Real Madrid,hombre,123,1,1,0,0,1


In [226]:
# Hay 11 casos donde el Alcance sale a 0 en twitter, habiendo interacciones (incoherente). Asumimos que hay un error en twitter
# y decidimos eliminar esos 11 registros para que no desvirtuen la métrica.

merged_df['exito_tweet'] = (merged_df['repercusion_twitter']/merged_df['alcance'])*1000

In [230]:
indices_a_eliminar = list(merged_df[(merged_df['alcance'] == 0) & (merged_df['repercusion_twitter'] > 0)].index)
merged_df.drop(index=indices_a_eliminar, inplace=True)

In [232]:
merged_df['exito_tweet'] = (merged_df['repercusion_twitter']/merged_df['alcance'])*1000

In [234]:
merged_df['exito_tweet'].value_counts().sort_index(ascending=False)

1333.333333       1
333.333333        1
200.000000        1
181.818182        1
142.857143        1
               ... 
0.179739          1
0.179083          1
0.178811          1
0.149365          1
0.000000       1119
Name: exito_tweet, Length: 2785, dtype: int64

In [285]:
merged_df.rename(columns={'alcance': 'alcance_twitter', 'likes': 'likes_twitter', 'rt':'retweets', 'respuestas':'respuestas_twitter'}, inplace=True)


In [292]:
merged_df.columns

Index(['web', 'titular', 'redactor', 'comentarios', 'link', 'noticia',
       'seccion', 'fecha_publicacion', 'fecha_actual', 'desactualizacion',
       'equipo', 'genero_redactor', 'tweets', 'alcance_twitter',
       'likes_twitter', 'retweets', 'respuestas_twitter',
       'repercusion_twitter', 'exito_tweet'],
      dtype='object')

In [291]:
merged_df = merged_df[['web', 'titular', 'redactor', 'comentarios', 'link', 'noticia',
       'seccion', 'fecha_publicacion', 'fecha_actual', 'desactualizacion',
       'equipo', 'genero_redactor', 'tweets','alcance_twitter',
       'likes_twitter', 'retweets', 'respuestas_twitter',
       'repercusion_twitter', 'exito_tweet']]

In [293]:
merged_df.to_excel('../data/repercusion_noticias_deportivas.xlsx', index=False)

In [286]:
merged_df.head()

,web,titular,redactor,comentarios,link,noticia,seccion,fecha_publicacion,fecha_actual,desactualizacion,equipo,genero_redactor,alcance_twitter,tweets,likes_twitter,retweets,respuestas_twitter,repercusion_twitter,exito_tweet
0,Mundo D.,Robo de bicicletas antes de comenzar el cicloc...,Agencias,1,https://www.mundodeportivo.com//ciclismo/20230...,Noticia,ciclismo,2023-01-22 00:00:00,2023-01-23,1,otro,desconocido,0,0,0,0,0,0,NaN
1,Mundo D.,"Jon Rahm, nº1 en FedEx Cup pero nº3 de un polé...",Redaccion,1,https://www.mundodeportivo.com//golf/20230123/...,Noticia,golf,2023-01-23 00:00:00,2023-01-23,0,otro,desconocido,0,0,0,0,0,0,NaN
2,Mundo D.,"Jon Rahm: ""Aprendí de la ética de trabajo de K...",Redaccion,0,https://www.mundodeportivo.com//golf/20230123/...,Noticia,golf,2023-01-23 00:00:00,2023-01-23,0,otro,desconocido,0,0,0,0,0,0,NaN
3,Mundo D.,"Los resultados de hoy lunes, octavos de final ...",Mundodeportivo.Com,0,https://www.mundodeportivo.com//tenis/open-aus...,Noticia,tenis,2023-01-23 00:00:00,2023-01-23,0,otro,desconocido,0,0,0,0,0,0,NaN
4,Mundo D.,26-28: Primera derrota de España en el Mundial...,Lluis,0,https://www.mundodeportivo.com//balonmano/2023...,Noticia,balonmano,2023-01-22 00:00:00,2023-01-23,1,otro,hombre,0,0,0,0,0,0,NaN


In [266]:
dimensiones = ['web','seccion', 'equipo', 'genero_redactor']
metricas = ['comentarios', 'alcance', 'tweets', 'likes', 'rt', 'respuestas', 'repercusion_twitter', 'exito_tweet']

In [282]:
dimension = input("Introduce una dimensión sobre la que agrupar: ")
desagrupa = input("Introduce una dimensión sobre la que desagrupar: ")
metrica = input("Introduce una métrica a visualizar: ")

Introduce una dimensión sobre la que agrupar: noticia
Introduce una dimensión sobre la que desagrupar: seccion
Introduce una métrica a visualizar: rt


In [284]:
# Agrupar por la columna "web" y "seccion" y sumar los comentarios
df_suma = df.groupby([dimension, desagrupa])[metrica].sum().reset_index()

# Crear el gráfico de barras
fig = go.Figure()

for seccion in df_suma[desagrupa].unique():
    df_seccion = df_suma[df_suma[desagrupa] == seccion]
    fig.add_trace(go.Bar(
        x=df_seccion[dimension],
        y=df_seccion[metrica],
        name=seccion,
        offsetgroup=seccion,
        width=0.5
    ))

# Agregar título y etiquetas de los ejes
fig.update_layout(
    title='Suma de comentarios acumulados por web',
    xaxis_title=dimension,
    yaxis_title=f'suma de {metrica}',
    barmode='stack'
)

# Mostrar el gráfico
fig.show()


In [ ]:
elif app_mode == '💻 Webs':

    x = 'web'
    y = st.sidebar.selectbox('Visibilidad según:', ['genero_redactor', 'seccion', 'equipo', 'repercusion'])

    st.title('💻 Visibilidad por WEB')

    condition = st.text_input("Condición de filtrado (ejemplo: comentarios > 100 & seccion == 'balonceso'):")
    if condition:
        filtered_df = df.query(condition)[df.columns]
    else:
        filtered_df = df.copy()
    st.write("Noticias totales:", filtered_df.shape[0])
    
    if y != 'repercusion':

        st.markdown('######')
        st.write('#### Noticias en primera plana por Diario Digital:')
        st.markdown('#####')

        data = filtered_df.copy()
        data = pd.crosstab(data[x], data[y])
        data = data.reset_index()

        fig = plt.figure(figsize=(200, 80))
        fig = px.bar(data, x=x, y=data.columns)
        st.plotly_chart(fig, use_container_width=True)
        
    else:
        
        z = st.sidebar.selectbox('Métrica:', ['comentarios', 'alcance', 'tweets', 'likes', 'rt', 'respuestas', 'repercusion_twitter', 'exito_tweet'])
        
        st.markdown('######')
        st.write('#### Repercusión de las noticias por Diario Digital:')
        st.markdown('#####')
        
        data = filtered_df.copy()
        df_suma = data.groupby(['web', y])[z].sum().reset_index()

        fig = go.Figure()

        for seccion in df_suma[y].unique():
            df_seccion = df_suma[df_suma[y] == seccion]
            fig.add_trace(go.Bar(
                x=df_seccion['web'],
                y=df_seccion[z],
                name=seccion,
                offsetgroup=seccion,
                width=0.5
            ))
            
        fig.update_layout(
        title='Suma de comentarios acumulados por web',
        xaxis_title='web',
        yaxis_title=f'suma de {z}',
        barmode='stack'
        )
        
        st.plotly_chart(fig, use_container_width=True)
